In [39]:
import sys
import pandas as pd
import sklearn
import tensorflow as tf

print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pd.__version__))
print('Sci-Kit Learn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))

Python: 3.8.0 (v3.8.0:fa919fdf25, Oct 14 2019, 10:23:27) 
[Clang 6.0 (clang-600.0.57)]
Pandas: 1.1.5
Sci-Kit Learn: 0.23.2
Keras: 2.4.0


In [76]:
f = 'data/autism-child-data.txt'

data = pd.read_table(f,sep=",",index_col=None)

In [77]:
# print shape of dataframe
print("Data Shape: {}".format(data.shape))
print(data.loc[0])

Data Shape: (292, 21)
A1_Score                      1
A2_Score                      1
A3_Score                      0
A4_Score                      0
A5_Score                      1
A6_Score                      1
A7_Score                      0
A8_Score                      1
A9_Score                      0
A10_Score                     0
age                           6
gender                        m
ethnicity                Others
jundice                      no
austim                       no
contry_of_res            Jordan
used_app_before              no
result                        5
age_desc           '4-11 years'
relation                 Parent
Class/ASD                    NO
Name: 0, dtype: object


# This is small dataset, so we may only be able to achieve a certain level of accuracy from the get-go. Let's look at the data some more...


In [78]:
data.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,0,0,1,1,0,1,0,0,...,m,Others,no,no,Jordan,no,5,'4-11 years',Parent,NO
1,1,1,0,0,1,1,0,1,0,0,...,m,'Middle Eastern ',no,no,Jordan,no,5,'4-11 years',Parent,NO
2,1,1,0,0,0,1,1,1,0,0,...,m,?,no,no,Jordan,yes,5,'4-11 years',?,NO
3,0,1,0,0,1,1,0,0,0,1,...,f,?,yes,no,Jordan,no,4,'4-11 years',?,NO
4,1,1,1,1,1,1,1,1,1,1,...,m,Others,yes,no,'United States',no,10,'4-11 years',Parent,YES
5,0,0,1,0,1,1,0,1,0,1,...,m,?,no,yes,Egypt,no,5,'4-11 years',?,NO
6,1,0,1,1,1,1,0,1,0,1,...,m,White-European,no,no,'United Kingdom',no,7,'4-11 years',Parent,YES
7,1,1,1,1,1,1,1,1,0,0,...,f,'Middle Eastern ',no,no,Bahrain,no,8,'4-11 years',Parent,YES
8,1,1,1,1,1,1,1,0,0,0,...,f,'Middle Eastern ',no,no,Bahrain,no,7,'4-11 years',Parent,YES
9,0,0,1,1,1,0,1,1,0,0,...,f,?,no,yes,Austria,no,5,'4-11 years',?,NO


In [79]:
data.describe()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,result
count,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000
mean,0.633562,0.534247,0.743151,0.551370,0.743151,0.712329,0.606164,0.496575,0.493151,0.726027,6.239726
std,0.482658,0.499682,0.437646,0.498208,0.437646,0.453454,0.489438,0.500847,0.500811,0.446761,2.284882
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,6.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000


In [80]:
data.dtypes

A1_Score            int64
A2_Score            int64
A3_Score            int64
A4_Score            int64
A5_Score            int64
A6_Score            int64
A7_Score            int64
A8_Score            int64
A9_Score            int64
A10_Score           int64
age                object
gender             object
ethnicity          object
jundice            object
austim             object
contry_of_res      object
used_app_before    object
result              int64
age_desc           object
relation           object
Class/ASD          object
dtype: object

# We have some missing values in here (age is listed as an object), so we'll need to go through this data and pre-process it. 

# We also have some redundant features here (age_description gives us no additional information about age), so we're going to drop some of these columns before passing it into our machine-learning model

In [81]:
data.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,0,0,1,1,0,1,0,0,...,m,Others,no,no,Jordan,no,5,'4-11 years',Parent,NO
1,1,1,0,0,1,1,0,1,0,0,...,m,'Middle Eastern ',no,no,Jordan,no,5,'4-11 years',Parent,NO
2,1,1,0,0,0,1,1,1,0,0,...,m,?,no,no,Jordan,yes,5,'4-11 years',?,NO
3,0,1,0,0,1,1,0,0,0,1,...,f,?,yes,no,Jordan,no,4,'4-11 years',?,NO
4,1,1,1,1,1,1,1,1,1,1,...,m,Others,yes,no,'United States',no,10,'4-11 years',Parent,YES
5,0,0,1,0,1,1,0,1,0,1,...,m,?,no,yes,Egypt,no,5,'4-11 years',?,NO
6,1,0,1,1,1,1,0,1,0,1,...,m,White-European,no,no,'United Kingdom',no,7,'4-11 years',Parent,YES
7,1,1,1,1,1,1,1,1,0,0,...,f,'Middle Eastern ',no,no,Bahrain,no,8,'4-11 years',Parent,YES
8,1,1,1,1,1,1,1,0,0,0,...,f,'Middle Eastern ',no,no,Bahrain,no,7,'4-11 years',Parent,YES
9,0,0,1,1,1,0,1,1,0,0,...,f,?,no,yes,Austria,no,5,'4-11 years',?,NO


# We're going to drop, age_description, results (which is a weighted average of their responses as a score). We're going to use our ML network decide that and we don't want to bias the network...

In [82]:
data = data.drop(['result','age_desc'], axis=1)

In [83]:
data.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,relation,Class/ASD
0,1,1,0,0,1,1,0,1,0,0,6,m,Others,no,no,Jordan,no,Parent,NO
1,1,1,0,0,1,1,0,1,0,0,6,m,'Middle Eastern ',no,no,Jordan,no,Parent,NO
2,1,1,0,0,0,1,1,1,0,0,6,m,?,no,no,Jordan,yes,?,NO
3,0,1,0,0,1,1,0,0,0,1,5,f,?,yes,no,Jordan,no,?,NO
4,1,1,1,1,1,1,1,1,1,1,5,m,Others,yes,no,'United States',no,Parent,YES
5,0,0,1,0,1,1,0,1,0,1,4,m,?,no,yes,Egypt,no,?,NO
6,1,0,1,1,1,1,0,1,0,1,5,m,White-European,no,no,'United Kingdom',no,Parent,YES
7,1,1,1,1,1,1,1,1,0,0,5,f,'Middle Eastern ',no,no,Bahrain,no,Parent,YES
8,1,1,1,1,1,1,1,0,0,0,11,f,'Middle Eastern ',no,no,Bahrain,no,Parent,YES
9,0,0,1,1,1,0,1,1,0,0,11,f,?,no,yes,Austria,no,?,NO


# Next we'll split this dataset into training features and a target feature, with te X_trains being all there A scores, and the y-target (prediction) being Class/ASD

In [84]:
x = data.drop(['Class/ASD'], axis=1)
y = data['Class/ASD']

In [85]:
x.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,relation
0,1,1,0,0,1,1,0,1,0,0,6,m,Others,no,no,Jordan,no,Parent
1,1,1,0,0,1,1,0,1,0,0,6,m,'Middle Eastern ',no,no,Jordan,no,Parent
2,1,1,0,0,0,1,1,1,0,0,6,m,?,no,no,Jordan,yes,?
3,0,1,0,0,1,1,0,0,0,1,5,f,?,yes,no,Jordan,no,?
4,1,1,1,1,1,1,1,1,1,1,5,m,Others,yes,no,'United States',no,Parent
5,0,0,1,0,1,1,0,1,0,1,4,m,?,no,yes,Egypt,no,?
6,1,0,1,1,1,1,0,1,0,1,5,m,White-European,no,no,'United Kingdom',no,Parent
7,1,1,1,1,1,1,1,1,0,0,5,f,'Middle Eastern ',no,no,Bahrain,no,Parent
8,1,1,1,1,1,1,1,0,0,0,11,f,'Middle Eastern ',no,no,Bahrain,no,Parent
9,0,0,1,1,1,0,1,1,0,0,11,f,?,no,yes,Austria,no,?


# Next, we need to change these categorical features into something a machine learning model can understand. We can do that with one-hot notation

In [86]:
# This will convert all our categorical features into 1-0 notation.
X = pd.get_dummies(x)

In [87]:
X.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,contry_of_res_Syria,contry_of_res_Turkey,used_app_before_no,used_app_before_yes,relation_'Health care professional',relation_?,relation_Parent,relation_Relative,relation_Self,relation_self
0,1,1,0,0,1,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
1,1,1,0,0,1,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
2,1,1,0,0,0,1,1,1,0,0,...,0,0,0,1,0,1,0,0,0,0
3,0,1,0,0,1,1,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,1,0,0,0,1,0,0,0
5,0,0,1,0,1,1,0,1,0,1,...,0,0,1,0,0,1,0,0,0,0
6,1,0,1,1,1,1,0,1,0,1,...,0,0,1,0,0,0,1,0,0,0
7,1,1,1,1,1,1,1,1,0,0,...,0,0,1,0,0,0,1,0,0,0
8,1,1,1,1,1,1,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9,0,0,1,1,1,0,1,1,0,0,...,0,0,1,0,0,1,0,0,0,0


In [88]:
X.columns.values

array(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score',
       'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score',
       'age_10', 'age_11', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8',
       'age_9', 'age_?', 'gender_f', 'gender_m',
       "ethnicity_'Middle Eastern '", "ethnicity_'South Asian'",
       'ethnicity_?', 'ethnicity_Asian', 'ethnicity_Black',
       'ethnicity_Hispanic', 'ethnicity_Latino', 'ethnicity_Others',
       'ethnicity_Pasifika', 'ethnicity_Turkish',
       'ethnicity_White-European', 'jundice_no', 'jundice_yes',
       'austim_no', 'austim_yes', "contry_of_res_'Costa Rica'",
       "contry_of_res_'Isle of Man'", "contry_of_res_'New Zealand'",
       "contry_of_res_'Saudi Arabia'", "contry_of_res_'South Africa'",
       "contry_of_res_'South Korea'",
       "contry_of_res_'U.S. Outlying Islands'",
       "contry_of_res_'United Arab Emirates'",
       "contry_of_res_'United Kingdom'", "contry_of_res_'United States'",
       'contry_of_res_Afg

In [89]:
# convert the categorical responses for the Y-values too
Y = pd.get_dummies(y)

In [90]:
Y.loc[:10]

,NO,YES
0,1,0
1,1,0
2,1,0
3,1,0
4,0,1
5,1,0
6,0,1
7,0,1
8,0,1
9,1,0


# Now that all our vectors are one-hot encoded, we're ready to split the data set into training and test sets. We'll do this train-test split using SciKit Learn. 

In [91]:
from sklearn import model_selection

In [92]:
# We'll split the train-test sets 80%-20%, which is standard
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X,Y, test_size=0.2)

In [93]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(233, 96)
(59, 96)
(233, 2)
(59, 2)


# Now let's build the neural network, we'll make a builder function to do this...

In [94]:
from tensorflow.keras import Input, Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam

In [110]:
def build_ANN(train_data, target_data):
    model = Sequential()
    model.add(Input(shape=train_data.shape[1]))
    model.add(Dense(8, kernel_initializer='normal', activation=LeakyReLU(alpha=0.2)))
    model.add(Dense(4, kernel_initializer='normal', activation=LeakyReLU(alpha=0.2)))
    model.add(Dense(target_data.shape[1], activation='sigmoid')) # sigmoid for binary classification
    
    optim = Adam(lr=0.001)
    model.compile(loss="binary_crossentropy", optimizer=optim, metrics=['accuracy'])
    return model

In [111]:
model = build_ANN(X_train, Y_train)
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 8)                 776       
_________________________________________________________________
dense_31 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_32 (Dense)             (None, 2)                 10        
Total params: 822
Trainable params: 822
Non-trainable params: 0
_________________________________________________________________


# Now that we've built the model, let's train it on the data...

In [112]:
model.fit(X_train, Y_train, epochs = 100, batch_size=10, verbose=1)

Epoch 1/100
24/24 [==============================] - 0s 792us/step - loss: 0.6932 - accuracy: 0.4764
Epoch 2/100
24/24 [==============================] - 0s 690us/step - loss: 0.6897 - accuracy: 0.6695
Epoch 3/100
24/24 [==============================] - 0s 694us/step - loss: 0.6820 - accuracy: 0.6481
Epoch 4/100
24/24 [==============================] - 0s 662us/step - loss: 0.6676 - accuracy: 0.5923
Epoch 5/100
24/24 [==============================] - 0s 703us/step - loss: 0.6445 - accuracy: 0.6652
Epoch 6/100
24/24 [==============================] - 0s 704us/step - loss: 0.6196 - accuracy: 0.6609
Epoch 7/100
24/24 [==============================] - 0s 732us/step - loss: 0.5901 - accuracy: 0.7897
Epoch 8/100
24/24 [==============================] - 0s 879us/step - loss: 0.5616 - accuracy: 0.8155
Epoch 9/100
24/24 [==============================] - 0s 928us/step - loss: 0.5332 - accuracy: 0.8069
Epoch 10/100
24/24 [==============================] - 0s 906us/step - loss: 0.5052 - accura

# The model was surprisingly accuracy on the training data reaching 100%, but we'll need to see if the model can generalize to new information, and that the model is not overfit. We'll generate a classification report to see the what the true accuracy is on the test data...

In [118]:
predictions = model.predict_classes(X_test)

In [119]:
predictions

array([1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0])

In [124]:
print('Prediction Results for Neural Network: ')
print()
print(accuracy_score(Y_test['YES'],predictions),'% Accuracy')
print()
print(classification_report(Y_test['YES'], predictions))


Prediction Results for Neural Network: 

0.9322033898305084 % Accuracy

              precision    recall  f1-score   support

           0       0.88      1.00      0.94        29
           1       1.00      0.87      0.93        30

    accuracy                           0.93        59
   macro avg       0.94      0.93      0.93        59
weighted avg       0.94      0.93      0.93        59



# Surprisingly accurate at 93%.

# Strong F1-Scores greater than 90%. This neural network can thus be considered usable for autism screening. 

# Very reliable in predicting autism, very low false positives (precision) and very low false negatives (recall)

# To improve this model, we could add dropout to the ANN that would help break up any colinearity between our features. 